In [ ]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
sys.path.insert(1, sys.path[0] + '/src')
print(sys.path)
import pandas as pd
from src.robo import *
from src.utils import *
from src.calcEMA import *
from src.myenv import *
from src.train import *

tail = regression_times + 50
use_cols = ['open_time', 'close']

estimator = 'lightgbm'

In [ ]:
symbol = 'BTCUSDT'
experiment, model = load_model(symbol, estimator)  # cassification_experiment

In [ ]:
df = get_data(symbol, save_database=False, interval='1h', tail=tail)
df.to_csv('log_after_1st_get_data.csv', sep=';', index=False)
df

In [ ]:
df = calc_RSI(df)
df.to_csv('log_after_1st_calc_RSI.csv', sep=';', index=False)
df

In [ ]:
df.dropna(inplace=True)
df

In [ ]:
numeric_features = ['close', 'rsi']
df, _ = regresstion_times(df, numeric_features, regression_times)
df.to_csv('log_after_1st_regresstion_times.csv', sep=';')
df

In [ ]:
df.info()

In [ ]:
window = 14
close_price = 'close'
count = df.shape[0]
df.iloc[count-1:count]['rsi'] = 0


#print(df[['close', 'rsi']].tail())
# _df = df.tail(window + 10)[[close_price, 'rsi']].copy()
_df = df[[close_price, 'rsi']].tail(window+50).copy()
_count = _df.shape[0]
#print(_df)
_df['change'] = _df[close_price].diff()
_df['gain'] = _df.change.mask(_df.change < 0, 0.0)
_df['loss'] = -_df.change.mask(_df.change > 0, -0.0)
_df['avg_gain'] = rma(_df.gain.to_numpy(), window)
_df['avg_loss'] = rma(_df.loss.to_numpy(), window)

_df['rs'] = _df.avg_gain / _df.avg_loss
_df['rsi'] = 100 - (100 / (1 + _df.rs))
#print(_df)

#print(df.iloc[count-1:count]['rsi'])
#print(_df.iloc[_count-1:_count]['rsi'])

df.update(_df.iloc[_count-1:_count]['rsi'])

df.tail()

In [ ]:
cont = 0

In [ ]:
#while True:

# df = get_data(symbol, save_database=False, interval='1h', tail=tail)
_df = get_klines(symbol, interval='1h', max_date=None, limit=1)
_df.to_csv(f'log_after_{cont}_get_klines.csv', sep=';', index=False)

#df = pd.concat([df, _df]).drop_duplicates(keep='last')
if _df.index.isin(df.index):
    df.update(_df)
else:
    df = pd.concat([df, _df])   
df.to_csv(f'log_after_{cont}_merge.csv', sep=';', index=False)

df = calc_RSI(df, last_one=True)
df.to_csv(f'log_after_{cont}_calc_RSI.csv', sep=';', index=False)

df, _ = regresstion_times(df, numeric_features, regression_times, last_one=True)
df.to_csv(f'log_after_{cont}_regresstion_times.csv', sep=';', index=False)

df[['open_time'] + numeric_features].to_csv('log_experiment_data.log', sep=';', index=False)
df_predict = experiment.predict_model(model, df.tail(1))
df_predict.to_csv(f'log_after_{cont}_df_predict.csv', sep=';', index=False)

operacao = df_predict['prediction_label'].values[0]
if (operacao.startswith('SOBE') or operacao.startswith('CAI')):
    send_message(df_predict)
time.sleep(sleep_refresh)
cont += 1

In [ ]:
import pandas as pd
df1 = pd.DataFrame({'open_time': [1, 55, 32, 234], 'close': [3, 4, 5, 6], 'C': [5, 6, 34, 5], 'D': [7, 8, 54, 66], 'label': ['A', 'A', 'B', 'B']}, index=[0,1,2,3])
df1

In [ ]:
df2 = pd.DataFrame({'open_time': [55], 'close': [77]}, index=[1])
df2

In [ ]:
i = df2.iloc[df2.shape[0]-1:df2.shape[0]].index
i

In [ ]:
df2.index.isin(df1.index)

In [ ]:
_df = get_klines('BTCUSDT', interval='1h', max_date=None, limit=200)
_df

In [ ]:
from pycaret.classification.oop import ClassificationExperiment

ca = ClassificationExperiment()
setup = ca.setup(_df, target='symbol')

In [ ]:
best = setup.compare_models()

best

In [ ]:
setup.pull().index[0]

In [ ]:
import datetime 
datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") 

In [ ]:
import pandas as pd
import datetime
import os

df_resultado_simulacao = pd.DataFrame()
if (os.path.exists('resultado_simulacao.csv')):
    df_resultado_simulacao = pd.read_csv('resultado_simulacao.csv', sep=';')

result_simulado = {}
result_simulado['model_name'] = 'BTCUSDT_lr_SL_2.0_RT_1440_RPL_24_1'
result_simulado['data'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
result_simulado['symbol'] = 'BTCUSDT'
result_simulado['estimator'] = 'lightgbm'
result_simulado['train_size'] = 0.7 
result_simulado['start_train_date'] = '2023-06-01'
result_simulado['start_test_date'] = '2023-08-08'
result_simulado['regression_times'] = 24*30
result_simulado['times_regression_profit_and_loss'] = 24
result_simulado['stop_loss'] = 2.0
result_simulado['fold'] = 3
result_simulado['saldo_inicial'] = 100.00
result_simulado['saldo_final'] = 91.87

print(result_simulado)

df = pd.DataFrame.from_dict([result_simulado])

df_resultado_simulacao = pd.concat([df_resultado_simulacao, df], ignore_index=True)
df_resultado_simulacao.sort_values('saldo_final', inplace=True)

df_resultado_simulacao.to_csv('resultado_simulacao.csv', sep=';', index=False)

In [ ]:
from binance.client import Client
client = Client()
klines = client.get_historical_klines(symbol='BTCUSDT', interval='1m', start_str='2023-09-01', limit=1000)
klines



In [ ]:
import pandas as pd
pd.to_datetime(1695036000000, unit='ms')

In [ ]:
import pandas as pd


all_klines_cols = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume',
                   'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']

columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'number_of_trades', 'ignore', 'close_time']

df_klines = pd.DataFrame(data=klines, columns=all_klines_cols)[columns]
df_klines['open_time'] = pd.to_datetime(df_klines['open_time'], unit='ms')
df_klines['close_time'] = pd.to_datetime(df_klines['close_time'], unit='ms')

In [ ]:
df_klines

In [ ]:
download_data()


In [ ]:
all_data = get_data('BTCUSDT', save_database=False, interval='1h', tail=-1, columns=['open_time', 'close'])
all_data

In [ ]:
import numpy as np

at = all_data.tail(1).open_time.dt.strftime('%Y-%m-%d %H').values[0]


In [ ]:
list_of_elements = ['close','open','high','low','volume']
fix_it='close'

combinations_list = []
if len(list_of_elements) > 0:    
    for i in range(1, len(list_of_elements) + 1):
        a = combinations(list_of_elements, i)
        for s in a:
            res = ''
            for j in s:
                res += f'{j},'
            combinations_list.append(res[0:len(res) - 1])

combinations_list

In [ ]:
from itertools import combinations
import pandas as pd

comb = combinations(range(1, 25 + 1), 15)
cols = ['col_' + str(i) for i in range(1, 15 + 1)]
resultado = pd.DataFrame(list(comb), columns=cols)
resultado

In [ ]:
resultado.query('col_1 in (1,2,3,4) and col_15 in (22,23,24,25)')

In [ ]:
import pandas as pd
results = ['a','b','c','d','d','d','d','d',]

pd.DataFrame(results, columns=['status'])['status'].value_counts()

In [ ]:
import pandas as pd
import sys
print(sys.path)
sys.path.insert(0, sys.path[0].replace('/src/jupyter_nb', ''))
print(sys.path)
import src.utils as utils
import src.myenv as myenv
import logging

logging.basicConfig(
    level=logging.DEBUG,  # Set the minimum level to be logged
    format="%(asctime)s [%(levelname)s]: %(message)s",
    handlers=[logging.StreamHandler()])


df = pd.read_csv('../../data/ADAUSDT/ADAUSDT.csv', sep=";", index_col='open_time')
df['open_time'] = pd.to_datetime(df.index, unit='ms')
df

In [ ]:
df.sort_index(inplace=True)
df.index.duplicated().sum()

In [ ]:
all_data = utils.regression_PnL(
            df,
            label=myenv.label,
            diff_percent=1.0,
            max_regression_profit_and_loss=6,
            drop_na=True,
            drop_calc_cols=True,
            strategy=None)

all_data
            

In [ ]:
all_data['prediction_label'] = all_data['status']
all_data = utils.simule_trading_crypto(all_data, '2023-06-01', '2023-09-19', 100, 1.0)
all_data 

In [28]:
import sys
print(sys.path)
sys.path.insert(0, sys.path[0].replace('/src/jupyter_nb', ''))
print(sys.path)
import src.utils as utils
import src.myenv as myenv
import src.send_message as sm
import os
import glob
import pandas as pd



['/home/marcelo/des/mg_crypto_trader', '/home/marcelo/des/mg_crypto_trader', '/home/marcelo/des/mg_crypto_trader', '/home/marcelo/des/mg_crypto_trader/src/jupyter_nb', '/usr/lib64/python310.zip', '/usr/lib64/python3.10', '/usr/lib64/python3.10/lib-dynload', '', '/home/marcelo/des/mg_crypto_trader/.env/lib64/python3.10/site-packages', '/home/marcelo/des/mg_crypto_trader/.env/lib/python3.10/site-packages', '/home/marcelo/des/mg_crypto_trader/.env/lib64/python3.10/site-packages/IPython/extensions', '/home/marcelo/.ipython']
['/home/marcelo/des/mg_crypto_trader', '/home/marcelo/des/mg_crypto_trader', '/home/marcelo/des/mg_crypto_trader', '/home/marcelo/des/mg_crypto_trader', '/home/marcelo/des/mg_crypto_trader/src/jupyter_nb', '/usr/lib64/python310.zip', '/usr/lib64/python3.10', '/usr/lib64/python3.10/lib-dynload', '', '/home/marcelo/des/mg_crypto_trader/.env/lib64/python3.10/site-packages', '/home/marcelo/des/mg_crypto_trader/.env/lib/python3.10/site-packages', '/home/marcelo/des/mg_crypt

In [25]:
utils.get_account_balance()

{'operation_date': 1695606011210, 'balance': 100.99, 'params': nan}

In [29]:
filename = f'{myenv.datadir}/account_balance.dat'
params = {}
params['operation_date'] = int(datetime.datetime.now().timestamp() * 1000)
params['balance'] = 100.99
params['params'] = None
data = pd.DataFrame(data=[params], index=[0])
if os.path.exists(filename):
	base = pd.read_csv(filename, sep=';')
	data = pd.concat([base, data], ignore_index=True)
	data.sort_values('operation_date', inplace=True)
data.to_csv(filename, sep=';', index=False)

In [31]:
symbol = 'BTCUSDT'
interval = '1h'
amount_invested = 100

params_operation = utils.get_latest_operation(symbol, interval)
if len(params_operation) > 0 and params_operation['operation'] == 'BUY':
	purchased = True
	purchase_price = float(params_operation['purchase_price'])
	take_profit = float(params_operation['take_profit'])
	stop_loss = float(params_operation['stop_loss'])
	purchase_operation = params_operation['status']
	amount_invested = params_operation['amount_invested']
	rsi = 0
	
	open_time = pd.to_datetime(params_operation["operation_date"], unit='ms')
	msg = f'Compra: Symbol: {symbol} - open_time: {open_time.strftime("%Y-%m-%d %H:%M:%S")} - Operação: {purchase_operation} - Valor Comprado: {purchase_price:.4f} - \
RSI: {rsi:.2f} - PnL: $ {amount_invested:.2f}'
	print(msg)
	print("************ COMPRADO ************")

Compra: Symbol: BTCUSDT - open_time: 2023-09-25 01:47:32 - Operação: CAI_1.0 - Valor Comprado: 26248.1699 - RSI: 0.00 - PnL: $ 100.00
************ COMPRADO ************


In [ ]:
res = utils.get_latest_operation('BTCUSDT', '1m')
len(res)

In [ ]:
filename = f'{myenv.datadir}/account_balance.dat'
if os.path.exists(filename):
	data = pd.read_csv(filename, sep=';')
	data.sort_values('operation_date', inplace=True)
	print(data.tail(1).to_dict(orient='records')[0])

In [ ]:
t = utils.get_telegram_key()
print(t)
myenv.telegram_key.append(t)

In [ ]:
myenv.telegram_key

In [ ]:
import pandas as pd

# Sample datetime data
date_strings = ["2023-09-24 10:00:00", "2023-09-24 15:30:00"]
dates = pd.to_datetime(date_strings)

# Create a Pandas Series or DataFrame with the datetime index
# Localize it to the Brazil/Brasilia time zone
dates_localized = dates.tz_localize('Brazil/East')

# Print the result
print(dates_localized)

In [ ]:
from binance.client import Client
client = Client()
time = client.get_server_time()
ix = pd.to_datetime(time['serverTime'], unit='ms')
ix

In [ ]:
data = pd.DataFrame(data = client.get_klines(symbol='BTCUSDT', interval='5m', limit=5), columns=myenv.all_klines_cols)
data['open_time'] = pd.to_datetime(data['open_time'], unit='ms')
data['close_time'] = pd.to_datetime(data['open_time'], unit='ms')
data

In [ ]:
data = utils.get_klines(symbol='BTCUSDT', interval='1h', limit=2, max_date=None, columns=myenv.all_klines_cols, parse_data=True)
data.iloc[0:1]

In [ ]:
data

In [ ]:
data.iloc[0:1]['open_time'].values[0]

In [ ]:
date = utils.get_latest_close_time('1h')
date

In [ ]:
#if ix.minute == 0:
ix = ix.replace(hour=ix.hour-1, minute=00, second=00, microsecond=00)
ix

In [ ]:
for i in range(0, 61):
	if  i % 5 == 0:
		print(i)

In [ ]:
ix = ix.replace(hour=14, minute=52, second=32, microsecond=0)
interval = 1

print(ix)
_ix_aux = ix.replace(hour=0, minute=0, second=0, microsecond=0)
day = ix.day-interval
for i in range(day, -1, -1):
	#print(f'i: {i} - min: {min} - rest: {i % interval}')
	if  (i <= day) and(i % interval == 0):
		print(f'hour: {i}')
		_ix_aux = _ix_aux.replace(day=i, hour=0, minute=0, second=0, microsecond=0)
		break
print(f'Last Close Time: {_ix_aux}')


In [ ]:
0 % 30

In [ ]:
data = utils.get_data('BTCUSDT', columns=myenv.all_cols)
data

In [ ]:
data.loc[ix]

In [ ]:
file_list = glob.glob(os.path.join(f'{myenv.datadir}/', 'resultado_simulacao_*.csv'))
top_params = []
columns = None
df_top_params = pd.DataFrame()
for file_path in file_list:
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path, sep=';')
        df_top_params = pd.concat([df_top_params, df.tail(1)], ignore_index=True)


df_top_params.to_csv(f'{myenv.datadir}/top_params.csv', sep=';', index=False)
return df_top_params.to_dict(orient='records')

In [ ]:
import pandas as pd
df = pd.DataFrame({'a': [1,674,3,674], 'b': [3,2,4,1], 'c': [23,35,7,78], 'd': [878,432,66,88]})
df

In [ ]:
df.sort_values(['a', 'b'],  ascending=[False, True])

In [ ]:
df.insert(0, 'dd', [212,4,52,2])
df

In [ ]:
df['tt'] = 'adf,fasd,gasdf,asdf'
df.loc[0, 'tt'] = 'as,f,as,fd,asd,f,asd,f,asd,f,as,df,'
df['c_tt'] = df['tt'].apply(lambda x: len(x.split(',')))
df

In [ ]:
utils.prepare_best_params()

In [ ]:
top_params

In [ ]:
symbol = 'BTCUSDT'
it = '5m'
filename = f'{myenv.datadir}/{symbol}/{symbol}_{it}'
df = pd.read_csv(f'{filename}.csv', sep=';')
# df.info()
df.to_csv(f'{filename}.dat', sep=';', index=False, compression=dict(method='zip'))
print(f'Done! {filename}')

In [ ]:
df_simb = pd.read_csv(f'{myenv.datadir}/symbol_list.csv')
for symbol in df_simb['symbol']:
	for it in ['1h', '5m']:
		filename = f'{myenv.datadir}/{symbol}/{symbol}_{it}'
		df = pd.read_csv(f'{filename}.csv', sep=';')
		# df.info()
		df.to_csv(f'{filename}.dat', sep=';', index=False, compression=dict(method='zip'))
		print(f'Done! {filename}')

In [ ]:
filename = f'{myenv.datadir}/BTCUSDT/BTCUSDT_5m'
df = pd.read_csv(f'{filename}.dat', sep=';', compression=dict(method='zip'))
df.info()
df
#df.to_csv(f'{filename}.csv', sep=';', index=False, compression=dict(method='zip', archive_name=f'{filename}.csv'))

In [ ]:
import datetime
import pandas as pd

now = datetime.datetime.now()
ordinal = int(now.timestamp() * 1000)
print(now)
print(ordinal)
print(pd.to_datetime(ordinal, unit='ms'))